In [10]:
import pandas as pd
import numpy as np

import sys
import os

sys.path.append( "../..")

from global_config import config

results_dir = config.get_property("results_dir")
data_dir    = config.get_property("data_dir")

data_dir

'/Users/chaosdonkey06/Dropbox/SantosVega-Lab/dengue_eco_evo/data'

In [12]:
import requests

sero_urls = [f"https://raw.githubusercontent.com/nextstrain/dengue/main/example_data/sequences_denv{z}.fasta.zst" for z in range(1, 4+1)]

for z, sero_url in enumerate(sero_urls):
    r = requests.get(sero_url)

    fasta_zst_file = os.path.join(data_dir, "nextstrain", f"sequences_denv{z}.fasta.zst")
    with open(fasta_zst_file, 'wb') as f:
    #giving a name and saving it in any required format
        f.write(r.content)

    # uncompress fasta files
    os.system(f"zstd -d {fasta_zst_file}")


/Users/chaosdonkey06/Dropbox/SantosVega-Lab/dengue_eco_evo/data/nextstrain/sequences_denv0.fasta.zst: 271250 bytes 
/Users/chaosdonkey06/Dropbox/SantosVega-Lab/dengue_eco_evo/data/nextstrain/sequences_denv1.fasta.zst: 304929 bytes 
/Users/chaosdonkey06/Dropbox/SantosVega-Lab/dengue_eco_evo/data/nextstrain/sequences_denv2.fasta.zst: 334582 bytes 
/Users/chaosdonkey06/Dropbox/SantosVega-Lab/dengue_eco_evo/data/nextstrain/sequences_denv3.fasta.zst: 163955 bytes 


<Response [200]>